In [ ]:
import json
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import yaml
from box import ConfigBox
from sklearn.metrics import f1_score
from tqdm import tqdm


### Set web serice URL

In [ ]:
# local testing
API_URL = "http://0.0.0.0:8080" 
# testing deployed service
# API_URL = "https://<YOUR_APP_NAME>.fly.dev" 

API_URL

### Load feature columns and data

In [ ]:
def load_params(params_path):
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
        params = ConfigBox(params)
    return params


proj_path = Path(os.getcwd()).parent.absolute()
params = load_params(proj_path/'params.yaml')
feat_cols = params.base.feat_cols
targ_col = params.base.targ_col
feat_cols, targ_col

In [ ]:
df = pd.concat([pd.read_csv(proj_path/'data'/'raw'/f'Churn_Modelling_{country}.csv') for country in ['France', 'Spain']])
df.head()

### Test `/predict` endpoint by sending one sample request

In [ ]:
my_obj = {
    "data": [
            {
      "CreditScore": 619,
      "Age": 42,
      "Tenure": 2,
      "Balance": 0,
      "NumOfProducts": 1,
      "HasCrCard": 1,
      "IsActiveMember": 1,
      "EstimatedSalary": 101348.88
            }
        ]
    }
x = requests.post(API_URL + '/predict', json = my_obj)
if x.ok:
    probs = x.json()
else:
    x.raise_for_status()

probs

### Create a functions for calling `/predict` endpoint

In [ ]:
def get_prob(obj):
    x = requests.post(API_URL + '/predict', json = obj)
    if x.ok:
        probs = x.json()
    else:
        x.raise_for_status()
    return probs

get_prob(my_obj)

### Check what would f1-scores look like if we send the data from the same geographies that the model was trained on 

In [ ]:
import time


def send_sample_requests(df):
    f1_score_list = []
    for _ in tqdm(range(50)):
        df_sample = df.sample(n=60)
        y_true = df_sample[targ_col].values
        obj = {"data": df_sample[feat_cols].to_dict('records')}
        probs = get_prob(obj)
        y_pred = np.array([prob < 0.5 for prob in probs]).astype(int)
        f1 = f1_score(y_true, y_pred)
        f1_score_list.append(f1)
        time.sleep(1)
    return f1_score_list
        
f1_score_list = send_sample_requests(df)
f1_score_list

### What would f1-scores look like for input data from new geography?

In [ ]:
df_germany =  pd.read_csv(proj_path/'data'/'more_data'/'Churn_Modelling_Germany.csv')
time.sleep(20) # this is to create a pause between requests. Later it'll be easier to distinguish the two types if requests
f1_score_list = send_sample_requests(df_germany)
f1_score_list


### ... as expected, the values are much lower.
### But, typically, we can't compute model metrics on production data right away 
### because ground truth labels might not be available until much-much later


### All we can do is try and look at "proxy" metrics that measure statistical differences (distances) between train data and production data

In [ ]:
x = requests.get(API_URL + '/drift_data')
if x.ok:
    data = x.json()
else:
    x.raise_for_status()

df_p_vals = pd.DataFrame(json.loads(data))
df_p_vals

### Values below the threshold (e.g. 0.05) indicate data drift

In [ ]:
import plotly_express as px

fig = px.line(df_p_vals, x='time', y=feat_cols)
fig.add_hline(y=0.05, line_color='red')